# Exploring the Dataset

In [18]:
import pandas as pd
import numpy as np

In [19]:
path='/Users/kefei/Documents/DSML/Introduction to Deep Learning/Second assignment/comp0090_1819-master/assignment_2/2_2_rnn_sequence_enconding/data/'

In [20]:
def read_data(path):
    text_file = open(path, "r")
    lines = text_file.readlines()
    
    x, y = [], []
    for line in lines:
        line_tokens = line.split(" ")
        y.append(int(line_tokens[0].replace("(","")))

        word_token_list = []
        for token in line_tokens:
            if token.endswith(')'):
                word_token_list.append(token.replace(')',''))
        x.append(word_token_list)
    return x,y

In [21]:
train_x, train_y = read_data(path+"train.sexp");
dev_x,   dev_y   = read_data(path+"dev.sexp");
test_x,  test_y  = read_data(path+"test.sexp");

# Word Representation

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
#vectorizer = CountVectorizer(analyzer = "word",   \
#                             tokenizer = callable,    \
#                             preprocessor = None, \
#                             stop_words = None) 
#train_data_features = vectorizer.fit_transform(train_x)
#train_data_features = train_data_features.toarray()

In [146]:
#only consider the top 1000 most popular words. 
vectorizer = CountVectorizer(tokenizer=lambda doc: doc, lowercase=False) #max_features=1000
cbow=vectorizer.fit_transform(train_x)

In [133]:
cbow = cbow.toarray()

In [187]:
### Make a matrix that stores the indices of each sentence up to 30 words. 
embedding = 30
index_matrix = np.zeros((len(train_x),embedding))
for i in range(len(train_x)):
    for j in range(min(len(train_x[i]),embedding)):
        word = train_x[i][j]
        index_matrix[i,j] = vectorizer.vocabulary_[str(word)] 

In [196]:
word_dict = vectorizer.vocabulary_

In [123]:
x = cbow1

In [120]:
CountVectorizer?

In [60]:
lengths = np.zeros(len(train_x))
for i in range(len(train_x)):
    lengths[i] = len(train_x[i])

In [61]:
lengths.max()

51.0

In [70]:
lengths.min()

1.0

In [193]:
y = train_y

# Data Analysis

You are expected to provide an analysis of the provided data. This can include word frequency analysis by class, character-level frequency analysis, analysis based on manual features (e.g. stopwords, negation words, punctuation), etc. Try to include some creative visualisations.

In [234]:
parameters =initialize_parameters(x)

(8672, 128)


In [228]:
LSTM_network = LSTM_forward(x,y,cbow,parameters,word_dict,index_matrix,batch_size=None,sentence_size=30)

ValueError: shapes (8544,18382) and (8672,128) not aligned: 18382 (dim 1) != 8672 (dim 0)

In [210]:
a = np.zeros((4,7))

In [212]:
a[,[2,3]]=1
a

array([[0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0.]])

# Recurrent Neural Network

Build a Recurrent Neural Network (RNN) to encode the sequence and predict each of the 5 classes. Tune hyper-parameters on the dev set. Evaluate performance in terms of accuracy on the test set.Bonus question: Compare performance on the train/dev/test sets - what does this tell you?

In [233]:
def initialize_parameters(x):
    hidden_size = 128 # Number of LSTM layer's neurons, a,f,i,o
    n = x.shape[0] # Number of input dimension == number of items in vocabulary
    W_size = hidden_size + n # Because we will concatenate LSTM state with the input
    
    
    #Maybe will concatenate Wf,Wi,Wc and Wo together to multiply in parallel
    Wf = np.random.randn(W_size, hidden_size) / np.sqrt( W_size/ 2.)
    Wi = np.random.randn(W_size, hidden_size) / np.sqrt( W_size/ 2.)
    Wa = np.random.randn(W_size, hidden_size) / np.sqrt( W_size/ 2.)
    Wo = np.random.randn(W_size, hidden_size) / np.sqrt( W_size/ 2.)
    Wy = np.random.randn(hidden_size, n) / np.sqrt(n / 2.)
    
    print(Wf.shape)
    
    bf=np.zeros((1, hidden_size))
    bi=np.zeros((1, hidden_size))
    ba=np.zeros((1, hidden_size))
    bo=np.zeros((1, hidden_size))
    by=np.zeros((1, n))
    
    parameters = {}
    parameters['Wf'] = Wf
    parameters['bf'] = bf
    parameters['Wi'] = Wi
    parameters['bi'] = bi
    parameters['Wa'] = Wa
    parameters['ba'] = ba
    parameters['Wo'] = Wo
    parameters['bo'] = bo
    parameters['Wy'] = Wy
    parameters['by'] = by
    
    return parameters


def initialize_network(hidden_size,T,batch_size):
    f = np.zeros((T,batch_size,hidden_size))
    i = np.zeros((T,batch_size,hidden_size))
    a = np.zeros((T,batch_size,hidden_size))
    o = np.zeros((T,batch_size,hidden_size))
    c = np.zeros((T,batch_size,hidden_size))
    h = np.zeros((T,batch_size,hidden_size))
    
    LSTM_network = {'f':f,'i':i,'a':a,'o':o,'c':c,'h':h}
    return LSTM_network


def initialize_grads(input_size,h_size):
    d_Wa = np.zeros(input_size, h_size)
    d_Wi = np.zeros(input_size, h_size)
    d_Wf = np.zeros(input_size, h_size)
    d_Wo = np.zeros(input_size, h_size)

    d_ba = np.zeros(h_size)
    d_bi = np.zeros(h_size)
    d_bf = np.zeros(h_size)
    d_bo = np.zeros(h_size)
    
    grads = {'d_Wa':d_Wa,'d_Wi':d_Wi,'d_Wf':d_Wf,'d_Wo':d_Wo,'d_ba':d_ba,'d_bi':d_bi,'d_bf':d_bf, 'd_bo':d_bo}
    return grads


#Input one current X or one sentence? 
#index matrix need to be adjusted with batch! 
def LSTM_forward(x,y,cbow,parameters,word_dict,index_matrix,batch_size=None,sentence_size=30):
    hidden_size=128
    D = cbow.shape[1]
    if not batch_size:
        batch_size = cbow.shape[0]
    
    LSTM_network = {}
    
    Wf, Wi, Wa, Wo, Wy = parameters['Wf'], parameters['Wi'], parameters['Wa'], parameters['Wo'], parameters['Wy']
    bf, bi, ba, bo, by = parameters['bf'], parameters['bi'], parameters['ba'], parameters['bo'], parameters['by']

    T = sentence_size #Number of LSTM cells? This is also the size of embedding..?
    network = initialize_network(hidden_size,T,batch_size)
    a,c,f,i,h = network['a'], network['c'],network['f'],network['i'],network['h']
    
    
    #Loop through each word in x. 
    for t in range(T):
        if t>0:
            h_prev, c_prev = h[t-1], c[t-1]
        else:
            h_prev, c_prev = np.zeros((batch_size,hidden_size)), np.zeros((batch_size,hidden_size))
            # Concatenate old state with current input
        
        #Need to specify xt
        index_t = index_matrix[:,t] #the t-th word of all the sentences in the batch.  
        X_one_hot = np.zeros((batch_size,D))
        
        
        for i in range(batch_size):
            X_one_hot[i,int(index_t[i])] = 1.
            
        X = np.column_stack((h_prev, X_one_hot))

        f[t] = sigmoid(X @ Wf + bf)
        print(f[t].shape)
        i[t] = sigmoid(X @ Wi + bi)
        o[t] = sigmoid(X @ Wo + bo)
        a[t] = tanh(X @ Wa + ba)

        c[t] = hf * c_prev + hi * a
        h[t] = ho * tanh(c)

    y_hat = h[T-1] @ Wy + by
    prob = softmax(y_hat) #Probably not softmax, as the scores are numerical. 

    #previous_outputs = [h, c] # Cache the states of current h & c for next iter
    cache = {} # Add all intermediate variables to this cache

    layers_name = ['x','a','i','f','c','h','y_hat']
    layers = [x,a,i,f,c,h,y_hat]
    
    for i in range(len(layers_name)):
        LSTM_network[layers_name[i]] = layers[i]
    
    return LSTM_network

def tanh(x):
    return np.tanh(x)

def sigmoid(x):
    return 1/(1+np.exp(-x))

def dtanh(x):
    return 1-np.tanh(x)**2

def MSE(y,y_hat):
    return (y - y_hat).T @ (y - y_hat)/(1.0*len(y))



## Backpropagation

In [ ]:
#Need to find a way to extract f_next... maybe no need
#backward for one particular layer t. 
#Do i update previous_outputs here with current h and c? 

def LSTM_backward(network,y,parameters, cost_func='MSE',sentence_size=30):
    #X is just a particular input X_t
    x_arr,y_hat, a_arr,c_arr,f_arr,h_arr,i_arr,o_arr= network['x'],network['y_hat'],network['a'],network['c'],network['f'],network['h'],network['i'],network['o']
    
    Wf, Wi, Wa, Wo, Wy = parameters['Wf'], parameters['Wi'], parameters['Wa'], parameters['Wo'], parameters['Wy']
    bf, bi, ba, bo, by = parameters['bf'], parameters['bi'], parameters['bc'], parameters['bo'], parameters['by']
    
    hidden_size = a.shape[0]
    
    grads = initialize_grads(input_size,hidden_size)
    grads_next_outputs = [np.zeros(hidden_size),np.zeros(hidden_size)]
    
    #T is the number of words in a sentence. size of embedding
    T = sentence_size
    
    if cost_func=='MSE':
        d_y = y - y_hat 
        
    d_Wy = h_arr.T @ d_y
        
    d_hT = d_y @ Wy.T
    
    for t in range(T)[::-1]:
        x,a,c,f,h,i,o = x_arr[t],a_arr[t],c_arr[t],f_arr[t],h_arr[t],i_arr[t],o_arr[t]
        
        if t>0:
            h_prev = h_arr[t-1]
            c_prev = c_prev[t-1]
        else:
            h_prev = np.zeros(hidden_size)
            c_prev = np.zeros(hidden_size)
        
        #Gradients of h and c*f stored for t+1
        d_h_next, d_c_next_times_f = grads_next_outputs[0],grads_next_outputs[1]
        
        inputs = [h_prev,x]
        
        d_h =  d_hT + d_h_next #### what is the dh thingie omg
        d_c =  o * d_h * dtanh(c) + d_c_next_times_f
        d_a =  d_c * i
        d_i =  d_c * a 
        d_f =  d_c * c_prev
        d_o = d_h * np.tanh(c) 

        #Stack those values together ...
        d_gates = [d_a*(1-a**2), d_i*i*(1-i),d_f*f*(1-f), d_o*o*(1-o)] #gradient for pre-non-linear for the gates
        W_gates =  [Wa, Wi, Wf, Wo]   
        d_inputs = sum[(W_gates[i].T * d_gates[i] for i in range(4))] #d_X contains both gradient for x and h_t-1. 

        #X is the input, X and h_prev
        #Do we only need to update the weights and biases? 

        d_Wa += inputs @ d_gates[0].T
        d_Wi += inputs @ d_gates[1].T
        d_Wf += inputs @ d_gates[2].T
        d_Wo = inputs @ d_gates[3].T

        d_ba += d_gates[0]
        d_bi += d_gates[1]
        d_bf += d_gates[2]
        d_bo += d_gates[3]    

        # Split the concatenated X, so that we get our gradient of h_prev
        d_h_next = d_inputs[:hidden_size, :] #Need to know how this is extracted exactly
        # Gradient for c_old in c = f * c_prev + hi * a, this case, c_prev becomes current c
        d_c_next_times_f = f * dc 

        grads_next_outputs[0] = d_h_next
        grads_next_outputs[1] = d_c_next_times_f 

        #previous_outputs[0] = h
        #previous_outputs[1] = c
        
        grads = {'Wa':d_Wa,'Wi':d_Wi,'Wf':d_Wf,'Wo':d_Wo,'ba':d_ba,'bi':d_bi,'bf':d_bf,'bo':d_bo}
    
    return grads


def update_parameters(parameters, grads, learning_rate):
    param_list = ['Wf','Wi','Wa','Wo','Wy','bf','bi','ba','bo','by']
    for param in param_list:  
        parameters[param] = parameters[param] - learning_rate * grads[param]
    return parameters


In [81]:
def train(iterations,x,y,learning_rate = 0.001,minibatch_size = 100, epsilon=1*10^(-3)):
    l = x.shape[0]
    n = x.shape[1]
    parameters = initialize_parameters()
    num_batches = int(l/minibatch_size) 
    batch_costs = np.zeros(num_batches) 
    average_costs = np.zeros(iterations) 
    prev_average_cost = 20000 
    i=1

    while (cost_diff> epsilon) & (i< iterations):
        startTime = datetime.now()
        #np.random.shuffle(x_train) #Shuffle the x_train array at each iteration
        for k in range(0, l, minibatch_size):
            batch_num = k/(minibatch_size)
            x_mini = x[k:k + minibatch_size]
            y_mini = y[k:k + minibatch_size]
        #model = initialize_network(h_size,T)
        grads = initialize_grads(input_size,h_size)
        model = LSTM_forward(x_mini,y_mini,parameters)
        loss = MSE(y_mini,model['y_hat'])
        grads = LSTM_backward(model,y_mini,parameters, cost_func='MSE')
        
    return model,loss

# Error Analysis

Perform a basic error analysis. What does the model predict well and what does it do less well on? Is there anything you can take away which could be used to improve the model in the future?

# Report 

You are expected to write a short (less than 400 words in total) report on: 

1.Data analysis - your key insights

2.Model decisions and tuning - RNN

3.Error analysis